<a href="https://colab.research.google.com/github/nujudaly/T5/blob/main/NUJUD_YOLO_Practical_Exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Road Traffic Vehicle Counting Exam

In this exam, you will use the YOLO (You Only Look Once) object detection model to analyze a video of road traffic. Your primary task is to count the number of vehicles exiting the road at each of the four exits (located at the top, bottom, left, and right of the frame). The video for this task is provided at `Datasets/YOLO_Exam_Video.mp4`.

## Objectives
- Load and process the video using OpenCV.
- Utilize the YOLO model to detect vehicles in each frame.
- Manipulate the video frames using OpenCV.
- Track vehicles as they move across the video frames.
- Count the number of vehicles exiting via the top street.
- Count the number of vehicles exiting via the bottom street.
- Count the number of vehicles exiting via the left street.
- Count the number of vehicles exiting via the right street.


## Setup Environment

Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [50]:
!pip install ultralytics

In [51]:
import cv2
import numpy as np
from ultralytics import YOLO, solutions
from ultralytics.utils.plotting import Annotator, colors
from shapely.geometry import Polygon
from shapely.geometry.point import Point

In [52]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Load the YOLO Model

You will first need to load the YOLO model, there is a custom model for this exam called `YOLO_Model.pt` under Datasets file ⚠️ PLEASE USE THIS MODEL AND DO NOT USE ANY OTHER MODEL ⚠️.
Write the code to load the YOLO model below:

In [53]:
model = YOLO("/content/drive/MyDrive/YOLO_Model.pt")

## Prepare the Video Capture

Create a variable to capture the video frames, you can use `cv2.VideoCapture()` to achive this.

In [54]:
video_frames = cv2.VideoCapture('/content/drive/MyDrive/YOLO_Video.mp4')
assert video_frames.isOpened(), "Error reading video file"

## Get Video Information

You can use `cv2` library to get these information fro the `VideoCapture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

In [55]:
w, h, fps = (int(video_frames.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

## Prepare Video Writer to Store the Output

Create a variable that uses `cv2.VideoCapture()` to save the video with the bounding boxes, specified Region of Interest rectangle (ROI) and the counted vehicles on each sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

In [56]:
video_writer = cv2.VideoWriter("YOLO_distance.avi", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

## Expermint to get the `(x1, y1, x2, y2)` of the ROI

### Hint
When defining the Regions of Interest (ROIs) for detecting vehicles exiting the road, consider the coordinates that outline the relevant areas. These coordinates can be specified as `(x1, y1, x2, y2)`, where `(x1, y1)` is the top-left corner and `(x2, y2)` is the bottom-right corner of the rectangle.

Experiment with different coordinates to accurately cover the streets where vehicles exit. Visualize these ROIs by drawing rectangles on a frame of the video and adjust the coordinates as needed until you capture the desired areas effectively.

In [57]:

#middle area where the flow of the cars on the highway should not be detecting those cars
#on left corner there are two exits we will detect cars when pass from them
#on right corner there are two an exit we will detect cars on
#the output should be a video frame with drawing rectangles on the exit area counting how many cars passes
#code of this cell is retrieved from: https://docs.ultralytics.com/guides/object-counting/#what-is-object-counting
#w =1920, h =1080

w, h, fps = (int(video_frames.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

print(f'{w, h}')

regions = [
    [(400, 540), (400, 540), (900, 136), (900, 136)],
    [(400, 810), (400, 810), (1300, 166), (1300, 166)],
    [(1300, 140), (1300, 140), (1900, 130), (1900, 136)],
    [(1300, 400), (1300, 400), (1300, 360), (1300, 360)]
]

video_writer = cv2.VideoWriter("object_counting_output.avi", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

counters = []
for i, region in enumerate(regions):
    counter = solutions.ObjectCounter(
        view_img=True,
        reg_pts=region,
        names=model.names,
        draw_tracks=True,
        line_thickness=2,
    )
    counters.append(counter)

while video_frames.isOpened():
    success, im0 = video_frames.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break

    results = model.track(im0, persist=True, show=False)

    for counter in counters:
        im0 = counter.start_counting(im0, results)

    video_writer.write(im0)

video_frames.release()
video_writer.release()
cv2.destroyAllWindows()


(1920, 1080)
WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

Polygon Counter Initiated.
WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

Polygon Counter Initiated.
WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

Polygon Counter Initiated.
WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

Polygon Counter Initiated.

0: 384x640 11 cars, 62.6ms
Speed: 3.3ms preprocess, 62.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 57.4ms
Speed: 4.2ms preprocess, 57.4ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 cars, 55.8ms
Speed: 4.2ms preprocess, 55.8ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 1 truck, 60.4ms
Speed: 4.3ms preprocess, 60.4ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 54.0ms
Speed: 5.0ms preprocess, 54.0ms inference, 1.9

## Process Video Frames and Identify Vehicles Left the Road Form Each Street

For each frame in the video, use the YOLO model to detect and track vehicles. You will need to write a loop that processes each frame and applies the YOLO model.

In each frame, after detecting the vehicles, determine whether they exited the road via the top, bottom, left, or right street. You can use the positions of the bounding boxes provided by YOLO to do this.

- The video should display bounding boxes around the detected objects.
- The video should display your name on the top.
- It should display the calculated center of each vehicle.
- The video should display the confidence score, alongside the object ID and class ID of each detected and tracked object.
- The video should display the rectangles representing the regions where you count the vehicles that have crossed and exited.
- The video should display the number of vehicles that exited via the top street.
- The video should display the number of vehicles that exited via the bottom street.
- The video should display the number of vehicles that exited via the left street.
- The video should display the number of vehicles that exited via the right street.

In [58]:

#middle area where the flow of the cars on the highway should not be detecting those cars
#on left corner there are two exits we will detect cars when pass from them
#on right corner there are two an exit we will detect cars on
#the output should be a video frame with drawing rectangles on the exit area counting how many cars passes
#code of this cell is retrieved from: https://docs.ultralytics.com/guides/object-counting/#what-is-object-counting
#w =1920, h =1080

w, h, fps = (int(video_frames.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

print(f'{w, h}')

regions = [
    [(400, 540), (400, 540), (900, 136), (900, 136)],
    [(400, 810), (400, 810), (1300, 166), (1300, 166)],
    [(1300, 140), (1300, 140), (1900, 130), (1900, 136)],
    [(1300, 400), (1300, 400), (1300, 360), (1300, 360)]
]

video_writer = cv2.VideoWriter("object_counting_output.avi", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

counters = []
for i, region in enumerate(regions):
    counter = solutions.ObjectCounter(
        view_img=True,
        reg_pts=region,
        names=model.names,
        draw_tracks=True,
        line_thickness=2,
    )
    counters.append(counter)

while video_frames.isOpened():
    success, im0 = video_frames.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break

    results = model.track(im0, persist=True, show=False)

    for counter in counters:
        im0 = counter.start_counting(im0, results)

    video_writer.write(im0)

cv2.putText(im0, "Nujud", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
video_frames.release()
video_writer.release()
cv2.destroyAllWindows()




(0, 0)
WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

Polygon Counter Initiated.
WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

Polygon Counter Initiated.
WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

Polygon Counter Initiated.
WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

Polygon Counter Initiated.


## Save and Submit Your Work

Notice that the counter is working in my other trails for the bounding boxes, but there is a distrortion in the preportions and with limited time it needs more trails.

Submit both the notebook and the output video